In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

=====================================================================

### Implementing the model to predict attrition in 5 mins,Note book is crisp and clear  if you are intrested know the description and conclusion you can refer my blog below.

Blog url:- https://www.devonblog.com/software-development/artificial-intelligence/attrition-prediction-using-machine-learning/

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, log_loss, classification_report)
import plotly.graph_objs as go
import plotly.offline as py

In [ ]:
# Reading the dataset
attritionData=pd.read_csv("/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")
attritionData.head(5)

### It's Quiet imbalance dataset, where 83.87% data have attrition label has "No" and only 16.12% data has got attrition rate has "yes".

In [ ]:
print((attritionData.Attrition.value_counts()/len(attritionData.index))*100)
sns.countplot(x='Attrition',data=attritionData)

### As per this ficiton datset we see people who "Travel Rarely" are the one who have quitted more,

In [ ]:
sns.countplot(x='BusinessTravel',hue='Attrition',data=attritionData,palette='RdBu_r')

### From the below line graph we can see people in the age group of 28 and 29 are more likely to quit than any other age group

In [ ]:
import plotly.express as px
attDataBasedonAge=attritionData.groupby(['Age','Attrition']).apply(lambda x:x['Age'].count()).reset_index(name='Counts')
px.line(attDataBasedonAge,x='Age',y='Counts',color='Attrition',title='Agewise Counts of People in an Organization')

### When we segregate based on Gender and attrition, we see female of age "29" group is having most attrition and male of age "28" group has most attrition than compared to any other age group.

In [ ]:
attDataBasedonAgeGender=attritionData.groupby(['Age','Attrition','Gender']).apply(lambda x:x['Age'].count()).reset_index(name='Counts')
attDataBasedonAgeGender['AttrGen']=attDataBasedonAgeGender.Attrition+"_"+attDataBasedonAgeGender.Gender
px.line(attDataBasedonAgeGender,x='Age',y='Counts',color='AttrGen',title='Age & Gender Counts of People in an Organization')


### As per the Graph we get to see Employees who work overtime are the one who likely to leave the orgnisation than compared to others.

In [ ]:
attDataBasedonOverTime=attritionData.groupby(['OverTime','Attrition']).apply(lambda x:x['OverTime'].count()).reset_index(name='Counts')

ax=sns.barplot(data=attDataBasedonOverTime.reset_index(), x='OverTime', y='Counts',hue='Attrition')
ax.set(xlabel='Overtime group Defined as "NO" and "YES" ', ylabel='Count of Each category')

### As per the Graph we see Employees who dont have stock option in the company are most likely to leave the organization.These employees can be a junior employees or mid level employees who might not have stock options

In [ ]:
attritionDataBasedonStock=attritionData.groupby(['StockOptionLevel','Attrition']).apply(lambda x:x['StockOptionLevel'].count()).reset_index(name='Counts')
attritionDataBasedonStock['StockTrueFalse']=attritionDataBasedonStock[['StockOptionLevel']].apply(lambda x:x.map({0:'No',1:'Yes',2:'Yes',3:'Yes'}))

ax=sns.barplot(data=attritionDataBasedonStock, x='StockTrueFalse', y='Counts',hue='Attrition',ci=None)
ax.set(xlabel='Stock Defined as "NO" and "YES" ', ylabel='Count of Each category')

### From the below Visualization we get to see that people who has job satisfaction level as "High" also has got hit attrition followed by "Low" job satisfaction level. This also indicates how the data was recorded for job satisfaction since if employee thinks it get tracked then they might shows the False postive indications.

In [ ]:
attritionDataBasedonJobSatisfaction=attritionData.groupby(['JobSatisfaction','Attrition']).apply(lambda x:x['JobSatisfaction'].count()).reset_index(name='Counts')

attritionDataBasedonJobSatisfaction['JobSatisfactionLevel']=attritionDataBasedonJobSatisfaction[['JobSatisfaction']].apply(lambda x:x.map({1:'Low',2:'Medium',3:'High',4:'Very High'}))
ax=sns.barplot(data=attritionDataBasedonJobSatisfaction, x='JobSatisfactionLevel', y='Counts',hue='Attrition',ci=None)
ax.set(xlabel='Stock Defined as "NO" and "YES" ', ylabel='Count of Each category')

### Below Heat or corelation map tells how the features are linked to other features. we can see
- Job level is directly proportional to Total working years.
- Montly Income is directly proportional to the Total working years.
- Stock option is Directly proptional to Job level.
- Performance Rating is Directly proportional to percentage salary hike.

In [ ]:
corr_matirx=attritionData.corr()
plt.figure(figsize=(20,12))
ax = sns.heatmap(corr_matirx[corr_matirx>0.4], annot = True, cmap="Set1")
top, bottom = ax.get_ylim()
ax.set_ylim(top+0.5, bottom-0.5)

### Feature Engineering
Lets encode categorical featues to build the predictive models

In [ ]:
attritionData01=attritionData.copy()
# Encoding the Target column
attritionData01['AttritionTarget']=attritionData01[['Attrition']].apply(lambda x:x.map({'Yes':1,'No':0}))

In [ ]:
# Empty list to store columns with categorical data

categoricalColumns = []
for col, value in attritionData01.iteritems():
    if value.dtype == 'object':
        categoricalColumns.append(col)

# Store the numerical columns in a list numerical
numericalColumns = attritionData01.columns.difference(categoricalColumns)

print("Total number of numerical columns are:- ",len(numericalColumns))
print("Total number of categorical columns are:- ",len(categoricalColumns))

In [ ]:
attrition_catgorical = attritionData01[categoricalColumns]
attrition_catgorical = attrition_catgorical.drop(['Attrition'], axis=1) # Dropping the target column
attrition_catgorical = pd.get_dummies(attrition_catgorical,drop_first=True)

attrition_numerical = attritionData01[numericalColumns]
attritionDatset_final = pd.concat([attrition_numerical, attrition_catgorical], axis=1)
attritionDatset_final.head(3)

In [ ]:
# Splitting the data for test and train
X = attritionDatset_final.drop("AttritionTarget",axis=1)   
y = attritionDatset_final["AttritionTarget"]

# Split data into train and test sets as well as for validation and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size= 0.80,random_state=0)

In [ ]:
# We still have imbalanceed data where the percentage of people stayin the organisation is more thant the percentage of the employee leaving the organisation
(y_train.value_counts()/len(y_train.index))*100


## We will use SMOTE technique to Balance out the data

In [ ]:
# we will use SMOTE technique to Balance out the data

sm = SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

# After balancing the data Here is how the Balanced percentage of Data looks like
(y_train_res.value_counts()/len(y_train_res.index))*100

### Building model using Random Forest(RF)

In [ ]:
seed = 0   # We set our random seed to zero for reproducibility
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000, 
    'max_features': 0.3,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 1
}

rf = RandomForestClassifier(**rf_params)
rf.fit(X_train_res, y_train_res)
rf_predictions = rf.predict(X_test)
print("Accuracy score: {}".format(accuracy_score(y_test, rf_predictions)))
print("="*80)
print(classification_report(y_test, rf_predictions))

### Finding the important Features/Columns from Random forest 

In [ ]:
trace = go.Scatter(
    y = rf.feature_importances_,
    x = attritionDatset_final.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = attritionDatset_final.columns.values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [ ]:
plt.figure(figsize=(15,40))
feat_importances = pd.Series(rf.feature_importances_, index=X.columns)
feat_importances.nlargest(len(X.columns)).sort_values().plot(kind='barh', align='center')

### Using XGBoost

In [ ]:
import xgboost
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=7, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [ ]:
classifier.fit(X_train_res,y_train_res)
prediction=classifier.predict(X_test)

print("Accuracy score: {}".format(accuracy_score(y_test, prediction)))
print("="*80)
print(classification_report(y_test, prediction))

In [ ]:
plt.figure(figsize=(15,40))
feat_importances = pd.Series(classifier.feature_importances_, index=X.columns)
feat_importances.nlargest(len(X.columns)).sort_values().plot(kind='barh', align='center')

#### Conclusion:-
- We were able to achieve 87% of accuracy on test data using XGboost.
- Please refer my blog for conclusion results.

https://www.devonblog.com/software-development/artificial-intelligence/attrition-prediction-using-machine-learning/